# Store Date Preparation

In [ ]:
import pandas as pd
from IPython.display import display, HTML
from sklearn import preprocessing

stores_file="../data/rossmann-store-sales/store.csv"
# read in file
df_stores=pd.read_csv(stores_file)

## Data Preparation

In [ ]:
## TODO rename PromoInterval to Promo2Interval
df_stores.rename(columns={"PromoInterval": "Promo2Interval"}, inplace=True)
## TODO ajust Data Types of Promo2 to Boolean
df_stores["Promo2"]=df_stores['Promo2'].astype('bool')
print(df_stores.info())
display(df_stores)

### Promo2 data cleaning

In [ ]:
# handel promo2
# locate the problem
print ("Value Count",  df_stores["Promo2"].value_counts())
print("Number of lines without promo2: ", df_stores[(df_stores["Promo2"] == False) & (df_stores["Promo2SinceWeek"].isna())].shape[0])
print("Number of lines with promo2 and Null values in Promo2Since: ", df_stores[(df_stores["Promo2"] == True) & (df_stores["Promo2SinceWeek"].isna())].shape[0])
# Set all to 0 when no Promo2

df_stores.fillna({"Promo2Interval":0}, inplace=True)
df_stores.fillna({"Promo2SinceWeek":0}, inplace=True)
df_stores.fillna({"Promo2SinceYear":0}, inplace=True)
df_stores.isna().mean()
## change data types
df_stores["Promo2SinceWeek"] = df_stores["Promo2SinceWeek"].astype(int)
df_stores["Promo2SinceYear"] = df_stores["Promo2SinceYear"].astype(int)
df_stores.dtypes

### Null values in Competition Distance cleaning

In [ ]:
# handel Null values in CompetitionDistance
print (df_stores.shape[0])
df_stores.dropna(subset=["CompetitionDistance"], inplace=True)
print (df_stores.shape[0])


## Binning the Competition Distance

In [ ]:

est = preprocessing.KBinsDiscretizer(n_bins=25, encode='ordinal').fit(df_stores[["CompetitionDistance"]])
df_stores["CompetitionDistanceBin"] = est.transform(df_stores[["CompetitionDistance"]])    
display(df_stores)

# calc mean for each bin and assign it to each instance
mean_dist = df_stores[["CompetitionDistance","CompetitionDistanceBin"]].groupby("CompetitionDistanceBin").mean().reset_index()
display(mean_dist)
df_stores = pd.merge(df_stores, mean_dist, how="inner", on="CompetitionDistanceBin")
display(df_stores)